First, create the model. This must match the model used in the interactive training notebook.

In [1]:
import torch
import torchvision

CATEGORIES = ['apex1']

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
model = model.cuda().eval().half()

Next, load the saved model.  Enter the model path you used to save.

In [2]:
model.load_state_dict(torch.load('final_racing_line.pth'))

Convert and optimize the model using ``torch2trt`` for faster inference with TensorRT.  Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme for more details.

> This optimization process can take a couple minutes to complete. 

In [3]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

Save the optimized model using the cell below

In [4]:
torch.save(model_trt.state_dict(), 'final_racing_line_trt.pth')

Load the optimized model by executing the cell below

In [5]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('final_racing_line_trt.pth'))

Create the racecar class

In [6]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

Create the camera class.

In [7]:
from jetcam.csi_camera import CSICamera


camera = CSICamera(width=224, height=224, capture_fps=65)


Finally, execute the cell below to make the racecar move forward, steering the racecar based on the x value of the apex.

Here are some tips,

* If the car wobbles left and right,  lower the steering gain
* If the car misses turns,  raise the steering gain
* If the car tends right, make the steering bias more negative (in small increments like -0.05)
* If the car tends left, make the steering bias more postive (in small increments +0.05)

In [8]:
import ipywidgets.widgets as widgets
import traitlets 
from IPython.display import display
controller = widgets.Controller(index=0)

display(controller)


Controller()

In [9]:
mode = widgets.IntSlider()
caption = widgets.Label(value='Mode')
user_mode_on = widgets.IntSlider(description='On')
user_mode_off = widgets.IntSlider(description='Off')

display(caption, user_mode_on, user_mode_off)

Label(value='Mode')

IntSlider(value=0, description='On')

IntSlider(value=0, description='Off')

In [10]:
user_on_button = traitlets.link((controller.buttons[11], 'value'), ( user_mode_on ,'value' ))
user_off_button = traitlets.link((controller.buttons[10], 'value'), ( user_mode_off ,'value' ))

In [11]:
%gui asyncio

In [12]:
import asyncio
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

In [13]:
def road_detection():
    while True:
        image = camera.read()
        image = preprocess(image).half()
        output = model_trt(image).detach().cpu().numpy().flatten()
        x = float(output[0])
        car.steering = x * STEERING_GAIN + STEERING_BIAS


In [14]:
from utils import preprocess
import numpy as np
import threading


In [15]:



STEERING_GAIN = 1
STEERING_BIAS = -0.1

async def f():
    while True:
        x = await wait_for_change(user_mode_on, 'value') 
        if user_mode_on.value == 1:
            car.throttle = 0.24
            xy = await wait_for_change(user_mode_off, 'value')
            if user_mode_off.value == 1:
                car.throttle = 0
                await asyncio.sleep(0.1)

thread1 = threading.Thread(target=road_detection)

        
    



In [16]:
try:
    thread1.start()
    asyncio.ensure_future(f())

except KeyboardInterrupt:
        car.throttle = 0
        print('Interrupted')

